In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import io

data_lines = []

with open("/content/IMDB Dataset.csv", "r", encoding="utf-8") as file:
    for line in file:
        data_lines.append(line)


csv_content = ''.join(data_lines)

try:
    imdb = pd.read_csv(io.StringIO(csv_content))
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")



In [ ]:
imdb.head()

In [ ]:
print("Number of rows: ", imdb.shape[0])
print("Nummer of columns: ", imdb.shape[1])

In [ ]:
imdb.info()

In [ ]:
imdb.sentiment.value_counts()

In [ ]:
imdb['review'][1]


In [ ]:
from bs4 import BeautifulSoup
cleantext = BeautifulSoup(imdb["review"][1], 'lxml').text
cleantext

In [ ]:
import re
cleantext = re.sub(r'[^\w\s]', '', cleantext)
cleantext

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stopwords.words('english')

In [ ]:
token = cleantext.lower().split()
stopword = set(stopwords.words('english'))
token_list = [ word for word in token if word.lower() not in stopword ]

In [ ]:
" ".join(token_list)

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('wordnet')

In [ ]:
nltk.download('omw-1.4')

In [ ]:
lemmatizer.lemmatize(" ".join(token_list))

In [ ]:
imdb.keys()

In [ ]:
from tqdm import tqdm
def data_cleaner(imdb):
    clean_data = []
    for review in tqdm(imdb):
        cleantext = BeautifulSoup(review, "lxml").text
        cleantext = re.sub(r'[^\w\s]', '', cleantext)
        cleantext = [ token for token in cleantext.lower().split() if token not in stopword ]
        cleantext = lemmatizer.lemmatize(" ".join(cleantext))
        clean_data.append(cleantext.strip())
    return clean_data

In [ ]:
clean_data = data_cleaner(imdb.review.values)

In [ ]:
clean_data[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imdb, imdb.sentiment, test_size=0.2, random_state=42, stratify=imdb.sentiment)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
le_test = LabelEncoder()
y_test = le_test.fit_transform(y_test)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
clean_data_train_data = data_cleaner(X_train.review.values)

In [ ]:
X_train['cleaned_text'] = clean_data_train_data
X_train.head()

In [ ]:
clean_data_test_data = data_cleaner(X_test.review.values)
X_test['cleaned_text'] = clean_data_test_data
X_test.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec = CountVectorizer()
vec = vec.fit(X_train.cleaned_text)
train_x_bow = vec.transform(X_train.cleaned_text)
test_x_bow = vec.transform(X_test.cleaned_text)

In [ ]:
print(train_x_bow.shape)
print(test_x_bow.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

In [ ]:
classifier = MultinomialNB()

In [ ]:
alpha_ranges = {
    "alpha": [0.001, 0.01, 0.1, 1, 10.0, 100]
}

In [ ]:
grid_search = GridSearchCV(classifier, param_grid=alpha_ranges, scoring='accuracy', cv=3, return_train_score=True)
grid_search.fit(train_x_bow, y_train)

In [ ]:
alpha = [0.001, 0.01, 0.1, 1, 10.0, 100]
train_acc = grid_search.cv_results_['mean_train_score']
train_std = grid_search.cv_results_['std_train_score']

test_acc = grid_search.cv_results_['mean_test_score']
test_std = grid_search.cv_results_['std_test_score']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(alpha, train_acc, label="Training Score", color='b')
plt.plot(alpha, test_acc, label="Cross Validation Score", color='r')

plt.title("Validation Curve with Naive Bayes Classifier")
plt.xlabel("Alpha")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.legend(loc = 'best')
plt.show()

In [ ]:
grid_search.best_estimator_

In [ ]:
classifier = MultinomialNB(alpha=1)
classifier.fit(train_x_bow, y_train)

In [ ]:
predict = classifier.predict(test_x_bow)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy is ", accuracy_score(y_test, predict))

In [ ]:
from sklearn.metrics import classification_report
print("Accuracy is ", classification_report(y_test, predict))

In [ ]:
text = ["I liked the movie. It was great"]
text_vec = vec.transform(text)
classifier.predict(text_vec)

In [ ]:
le.inverse_transform([1])

In [ ]:
text = ["Movie was worse"]
text_vec = vec.transform(text)
classifier.predict(text_vec)

In [ ]:
le.inverse_transform([0])